In [1]:
## Libraries

suppressMessages(install.packages("readxl"))
suppressMessages(install.packages("ggplot2"))
suppressMessages(install.packages("ggpubr"))
suppressMessages(install.packages("tidyr"))


library("readxl", warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(repr, warn.conflicts = FALSE)
library(ggpubr, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(caret)

package 'readxl' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'readxl'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem z kopiowaniem C:\Users\Kamil\Documents\R\win-library\4.0\00LOCK\readxl\libs\x64\readxl.dll do C:\Users\Kamil\Documents\R\win-library\4.0\readxl\libs\x64\readxl.dll: Permission denied"
Warning message:
"restored 'readxl'"



The downloaded binary packages are in
	C:\Users\Kamil\AppData\Local\Temp\RtmpO6eWAD\downloaded_packages
package 'ggplot2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Kamil\AppData\Local\Temp\RtmpO6eWAD\downloaded_packages
package 'ggpubr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Kamil\AppData\Local\Temp\RtmpO6eWAD\downloaded_packages
package 'tidyr' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'tidyr'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem z kopiowaniem C:\Users\Kamil\Documents\R\win-library\4.0\00LOCK\tidyr\libs\x64\tidyr.dll do C:\Users\Kamil\Documents\R\win-library\4.0\tidyr\libs\x64\tidyr.dll: Permission denied"
Warning message:
"restored 'tidyr'"



The downloaded binary packages are in
	C:\Users\Kamil\AppData\Local\Temp\RtmpO6eWAD\downloaded_packages


Loading required package: lattice



In [ ]:
data <- read_excel("data.xlsx")

# Wstępne przetwarzanie danych

- Dopisanie identyfikatorów pacjenta dla wierszy z NA
- Zamiana etykiet kolumn z greckimi literami
- Zamiana dat na numerable
- Usunięcie niepotrzebnych wskaźników (2019-nCoV nucleic acid detection). W przypadku drugiego parametru nie jestem pewny jak interpretować puste wartości. Wartości wypełnione to tylko -1.

In [ ]:
previous_id = 1;
loop_iterator = 1;

data <- data %>% rename(`Interleukin 1B` = `Interleukin 1ß`)
data <- data %>% rename(`Tumor necrosis factora` = `Tumor necrosis factorα`)
data <- data %>% rename(`y-glutamyl transpeptidase` = `γ-glutamyl transpeptidase`)

data <- data %>% 
        mutate(nRE_DATE = as.numeric(RE_DATE)) %>%
        mutate(`nAdmission time` = as.numeric(`Admission time`)) %>%
        mutate(`nDischarge time` = as.numeric(`Discharge time`))

data <- select(
            data, 
           -c(
               "Admission time",
               "Discharge time",
               "2019-nCoV nucleic acid detection"
           )
          )

for (record in data$PATIENT_ID) {
    
    if (is.na(record)) {
        data$PATIENT_ID[loop_iterator] = previous_id;
    } else {
        previous_id = record;
    }

    loop_iterator <- loop_iterator + 1;
}

summary(data)

# Graficzna analiza zależności parametrów od wyniku

W tym kroku, w celu zapoznania się lepiej z danymi, wygenerowałem dla każdego atrybutu wykres zależności outcome od tego atrybutu. Poniżej kilka, moim zdaniem, najciekawszych z nich wraz z komentarzem. 

#### Generowanie wszystkich wykresów
Blok został zakomentowany, ponieważ w sekcji poniżej pokazane są najciekawsze z wyników.

In [ ]:
options(repr.plot.width=16, repr.plot.height=10)

show <- function() {
    for (column_name in colnames(data)) {

        kv <- data %>%
                select(!!as.name(column_name), outcome) %>%
                filter(!is.na(!!as.name(column_name))) %>%
                count(!!as.name(column_name), outcome)

        print(ggplot(kv, aes(x=!!as.name(column_name), y=outcome, size = n))
              + geom_point(alpha=0.7)
              + ggtitle(paste("Zależnośc wyniku badania od atrybutu: ", column_name))
              + theme(text = element_text(size=20))
             )
    }
}

# show()

#### Najciekawsze (według mnie) wykresy

In [ ]:
options(repr.plot.width=16, repr.plot.height=40)
plots = list()
interesting_cols = c(
    "age",
    "gender",
    "Hypersensitive cardiac troponinI",
    "Prothrombin time",
    "albumin",
    "Direct bilirubin",
    "Total cholesterol",
    "Amino-terminal brain natriuretic peptide precursor(NT-proBNP)",
    "Lactate dehydrogenase",
    "neutrophils count"
)
i = 1

for (column_name in interesting_cols) {
    kv <- data %>%
                select(!!as.name(column_name), outcome) %>%
                filter(!is.na(!!as.name(column_name))) %>%
                count(!!as.name(column_name), outcome)

    plots[[i]] <- (ggplot(kv, aes(x=!!as.name(column_name), y=outcome, size = n))
          + geom_point(alpha=0.7)
          + ggtitle(paste("Zależnośc wyniku badania od atrybutu: ", column_name))
          + theme(text = element_text(size=15))
         )
    
    i = i + 1
}

ggarrange(
    plots[[1]],
    plots[[2]],
    plots[[3]],
    plots[[4]],
    plots[[5]],
    plots[[6]],
    plots[[7]],
    plots[[8]],
    plots[[9]],
    plots[[10]],
    ncol = 2, nrow = 5)

# Korelacja atrybutów
Lista kilku atrybutów, których korelacja w stosunku do atrubutu "outcome" jest największa

In [ ]:
korelacje = {}

for (column_name in colnames(select(data, -c("outcome", "PATIENT_ID", "RE_DATE")))) {
    kv <- data %>%
            select(!!as.name(column_name), outcome) %>%
            filter(!is.na(!!as.name(column_name)))
    
    korelacje[column_name] <- abs(cor(kv$outcome, kv %>% select(all_of(column_name))))
}

ordered <- korelacje[order(unlist(korelacje))]

options(repr.plot.width=16, repr.plot.height=6)
names <- names(rev(ordered))[1:6]

for (column_name in names) {
    kv <- data %>%
                select(!!as.name(column_name), outcome) %>%
                filter(!is.na(!!as.name(column_name))) %>%
                count(!!as.name(column_name), outcome)

    plot <- ggplot(kv, aes(x=!!as.name(column_name), y=outcome, size = n)) +
            geom_point() +
            geom_smooth(method="lm") +
            ggtitle(paste("Zależnośc wyniku badania od atrybutu: ", column_name, ". Korelacja: ", ordered[column_name])) + 
            theme(text = element_text(size=15))
    
    suppressMessages(print(plot))
}

# Zmiana parametrów w czasie
Wizualizacja zmiany współczynników krwi u pacjentów w czasie przebywania choroby ze skutkiem śmiertelnym. 

In [ ]:
dead <- data %>%
            filter(outcome == 1) %>%
            select(c(1, 2, 5:(length(data) - 3) ))
            

patients_data = NA
initialized = FALSE

for (patient_id in dead$PATIENT_ID) {
    patient_data <- dead %>%
                    filter(PATIENT_ID==patient_id) %>%
                    mutate(day=format(RE_DATE, format='%m/%d/%Y')) %>%
                    group_by(day, PATIENT_ID) %>%
                    summarise_each(funs(mean(., na.rm = TRUE))) %>%
                    arrange(desc(RE_DATE))
    
    patient_data$day = seq.int(nrow(patient_data))
    
    if (!initialized) {
        patients_data = patient_data
        initialized =TRUE
    } else {
        patients_data = union(patients_data, patient_data)
    }
    
}

max_day <- max(patients_data["day"])
patients_data <- patients_data %>% select(c(1, 2, 5:length(patients_data)))
patients_data <- patients_data %>% mutate(day=(max_day - day + 1))

In [ ]:
patients_data <- ungroup(patients_data)

Wyniki analizy przedstawiąją zmianę parametrów krwi przed zgonem. Skrajny prawy słupek oznacza współczynniki krwi u pacjenta w dniu zgonu (dla wielu pomiarów wyciągana jest średnia). Przeciętnie pacjenci trafiali do szpitala kilka dni przed zgonem, dlatego wyniki badań krwi pacjentów kilkanaście dni przed zgonem (lewa stronwa wykresu) są szczątkowe. 

Poniższy skrypt przedstawia sposób wygenerowania wszystkich wykresów. Podobnie jak w przypadku poprzedniej analizy, spośród wszystkich wybrałem kilka interesujących i przedstawię tylko te.

In [ ]:
options(repr.plot.width=16, repr.plot.height=10)
hide_cols = c("day", "PATIENT_ID")

show <- function(to_show) {
    for (column_name in to_show) {
        
        if (column_name %in% hide_cols) {
            next
        }

        kv <- patients_data %>%
                select(!!as.name(column_name), day) %>%
                filter(!is.na(!!as.name(column_name)))

        print(
                ggplot(kv, aes(x=day, y=!!as.name(column_name)))
              + geom_point()
              + ggtitle(paste("Zależnośc wyniku badania od atrybutu: ", column_name))
              + theme(text = element_text(size=20))
             )
    }
}

# show(colnames(patients_data))

In [ ]:
options(repr.plot.width=15, repr.plot.height=8)

show(c("creatinine", "Lactate dehydrogenase"))

Podobnej selekcji najciekawszysch wyników dokonałem także dla uwzględniając korelacje atrybutów. Zgodnie z tym współczynnikiem najbardziej interesujące atrybuty to:

In [ ]:
korelacje = {}
hide_cols = c("day", "PATIENT_ID")

for (column_name in colnames(patients_data)) {
    
    if (column_name %in% hide_cols) {
        next
    }
    
    kv <- patients_data %>%
            select(!!as.name(column_name), day) %>%
            filter(!is.na(!!as.name(column_name)))
    
    
    
    korelacje[column_name] <- abs(cor(kv$day, kv %>% select(all_of(column_name))))
}

ordered <- korelacje[order(unlist(korelacje))]

options(repr.plot.width=16, repr.plot.height=6)
names <- names(rev(ordered))[1:2]

for (column_name in names) {
    kv <- patients_data %>%
                select(!!as.name(column_name), day) %>%
                filter(!is.na(!!as.name(column_name)))

    plot <- ggplot(kv, aes(x=day, y=!!as.name(column_name))) +
            geom_point() +
            geom_smooth(method="lm") +
            ggtitle(paste("Zależnośc dnia od atrybutu: ", column_name, ".Korelacja: ", ordered[column_name])) + 
            theme(text = element_text(size=15))
    
    suppressMessages(print(plot))
}

# Klasyfikacja

In [ ]:
library(randomForest)

Pominięcie zbędnych kolum i zmiana wartości outcome

In [ ]:
classification_data <- data %>%
        select(-c(`nAdmission time`, `nDischarge time`, `RE_DATE`, `nRE_DATE`))

i = 1
for (row in classification_data$outcome) {
    if (row==1) {
        classification_data$outcome[i] = "dead"
    } else {
        classification_data$outcome[i] = "alive"
    }
    
    i = i + 1
}

classification_data <- classification_data %>% 
                        group_by(PATIENT_ID, outcome) %>%
                        summarise_each(funs(mean(., na.rm = TRUE))) %>%
                        ungroup() %>%
                        select(-c("PATIENT_ID"))

Podzielenie danych na 3 ziory (treningowy, walidacyjny i testowy)

In [ ]:
set.seed(23)

inTraining <- 
    createDataPartition(
        y = classification_data$outcome,
        p = .7,
        list = FALSE)

classification_data_training <- classification_data[ inTraining,]
nottraining  <- classification_data[-inTraining,]

inTesting <- 
    createDataPartition(
        y = nottraining$outcome,
        p = .5,
        list = FALSE)

classification_data_testing = nottraining[inTesting, ]
classification_data_validating = nottraining[-inTesting, ]

Uzupełnienie wartości pustych wartościami średnimi

In [ ]:
for(i in 4:ncol(classification_data_training)){
    mean <-  mean(classification_data_training[[i]], na.rm = TRUE)
    classification_data_training[is.na(classification_data_training[[i]]),i] <- mean
}

for(i in 4:ncol(classification_data_validating)){
    mean <-  mean(classification_data_validating[[i]], na.rm = TRUE)
    classification_data_validating[is.na(classification_data_validating[[i]]),i] <- mean
}

for(i in 4:ncol(classification_data_testing)){
    mean <-  mean(classification_data_testing[[i]], na.rm = TRUE)
    classification_data_testing[is.na(classification_data_testing[[i]]),i] <- mean
}

Uzupełnienie danych pacjentów 

In [ ]:
set.seed(128)

control <- trainControl(
    method="repeatedcv", 
    number=2, 
    repeats=2,
    allowParallel = TRUE)

df <- data.frame(set=character(),
                 ntree=integer(), 
                 mtry=integer(),
                 score=double(),
                 stringsAsFactors=FALSE) 

for (ntree in c(5, 10, 15, 20, 25, 30)){
    
    for (mtry in c(1:10)) {
        set.seed(123);
        
        fit <- train(
            outcome ~ .,
            data = classification_data_training,
            method = "rf",
            trControl = control,
            metric='Accuracy',
            tuneGrid = expand.grid(mtry=c(mtry)),
            ntree = ntree
        )
        
        rfClasses <- predict(fit, newdata = classification_data_validating)
        rtClasses <- predict(fit, newdata = classification_data_training)
        fm <- confusionMatrix(table(rfClasses, classification_data_validating$outcome))
        tm <- confusionMatrix(table(rtClasses, classification_data_training$outcome))
        
        new_row <- data.frame("validating", mtry, ntree, fm$overall['Accuracy'])
        names(new_row)<-c("set", "mtry", "ntry", "score")
        df <- rbind(df, new_row)
        
        new_row <- data.frame("training", mtry, ntree, tm$overall['Accuracy'])
        names(new_row)<-c("set", "mtry", "ntry", "score")
        df <- rbind(df, new_row)
    }
}

In [ ]:
dff <- df %>% mutate(grouped_id = row_number()) %>% spread('mtry', 'score')  %>% 
            select(-c("grouped_id")) %>%
            group_by(set, ntry) %>% 
            summarise_each(funs(mean(., na.rm = TRUE))) %>%
            ungroup()


plt <- ggplot(dff %>% filter(set=="training"), aes(x=ntry))

for (i in c(1:10)) {
    plt <- plt + 
        geom_line(aes(y=!!as.name(i))) +
        geom_point(aes(y=!!as.name(i)))

}
print(plt + theme(text = element_text(size=20)) + ggtitle("Wyniki dla zbioru treningowego"))



vall <- dff %>% filter(set=="validating")
plt <- ggplot(vall, aes(x=ntry))

for (i in c(1:10)) {
    plt <- plt + 
                geom_line(aes(y=!!as.name(i))) +
                geom_text(aes(x = 4, y = vall[[toString(i)]][[1]], label = paste("mtry = ", toString(i))), size=6) +
                geom_point(aes(y=!!as.name(i)))
}
print(plt + theme(text = element_text(size=20)) + ggtitle("Wyniki dla zbioru walidacyjnego"))


In [ ]:
fit <- train(
    outcome ~ .,
    data = classification_data_testing,
    method = "rf",
    trControl = control,
    metric='Accuracy',
    tuneGrid = expand.grid(mtry=c(3)),
    ntree = 30
)

### Miary klasyfikacji
Macierz pomyłek, pokazuje dokładnie (nie tylko procentowo) dokonane błędy

Skuteczność pozwala porównywać bezpośrednio 2 wyniki algorytmu jednym współczynnikiem

Pos Pred Value i Neg Pred Value - podobnie jak wyżej, jednak ze względu na specyfikę problemu, prawodopodobnie bardziej istotne będzie maksymalizowanie wskaźnika Pos Pred Value

In [ ]:
confusionMatrix(table(rfClasses, classification_data_testing$outcome), positive="dead")$table

confusionMatrix(table(rfClasses, classification_data_testing$outcome))$overall['Accuracy']
confusionMatrix(table(rfClasses, classification_data_testing$outcome))$byClass['Pos Pred Value']
confusionMatrix(table(rfClasses, classification_data_testing$outcome))$byClass['Neg Pred Value']

### Ważność współczynników

In [ ]:
varImp(fit)